In [2]:
from collections import defaultdict
import numpy as np

In [287]:
# CSA helpers

def search_next_departure(stop_departures, time):
    '''
        Search the first departure after moment time in the same stop through binary search in the list of S[stop]
    '''
    start = 0
    end = len(stop_departures) - 1

    next_departure_index = -1
    while start <= end:
        mid = (start + end) // 2;

        # Search in the right side
        if (stop_departures[mid]['departure_time'] < time):
            start = mid + 1

        # Search in the left side
        else:
            next_departure_index = mid
            end = mid - 1
     
    
    # Search is always successful because of the last np.inf departure
    next_profile_entry = stop_departures[next_departure_index]
        
    return next_departure_index, next_profile_entry

def shift(vector):
    # TODO: include max for earliest arrival
    new_vector = vector.copy()
    new_vector[1:] = vector[:-1]
    new_vector[0] = np.inf
    return new_vector

def minimize(vector1, vector2):
    return np.minimum(vector1, vector2)

def minimize_with_exits(vector_with_exits, tau_c, connection):
    indices_to_change = np.where(tau_c < vector_with_exits[0])[0]
    
    new_arrival_times = vector_with_exits[0].copy()
    new_arrival_times[indices_to_change] = tau_c[indices_to_change]
    new_exits = vector_with_exits[1].copy()
    for index in indices_to_change:
        new_exits[index] = connection
    return (new_arrival_times, new_exits)
    
def minimize_with_stops(old_arrivals, old_connections, tau_c, connection):
    indices_to_change = np.where(tau_c < old_arrivals)[0]
    
    new_arrival_times = old_arrivals.copy()
    new_arrival_times[indices_to_change] = tau_c[indices_to_change]
    new_connections = old_connections.copy()
    for index in indices_to_change:
        new_connections[index] = connection
    return (new_arrival_times, new_connections)

def minimize_with_enter_exit_connections(old_arrivals, old_enter_connections, tau_c, connection, old_exit_connections, trip_exit_connections):
    indices_to_change = np.where(tau_c < old_arrivals)[0]
    
    new_arrival_times = old_arrivals.copy()
    new_arrival_times[indices_to_change] = tau_c[indices_to_change]
    new_enter_connections = old_enter_connections.copy()
    new_exit_connections = old_exit_connections.copy()
    for index in indices_to_change:
        new_enter_connections[index] = connection
        new_exit_connections[index] = trip_exit_connections[index]
        
    return (new_arrival_times, new_enter_connections, new_exit_connections)

In [320]:
def CSA(timetable, target_stop, min_departure_time, max_arrival_time, max_connections):
    '''
        Implementation of Pareto Connection Scan profile algorithm with interstop footpaths (https://arxiv.org/pdf/1703.05997.pdf)
    '''
    # timetable
    stops, connections, trips, footpaths = timetable
    S = defaultdict(lambda: [{'departure_time': np.inf, 'arrival_times': np.ones(max_connections) * np.inf, 'enter_connections': [None] * max_connections, 'exit_connections': [None] * max_connections}]) # entries of the type       stop: [(departure_time, [arrival_time_with_1_connection, ..., arrival_time_with_max_connections], incoming_connection, [outgoing_connections])]
    T = defaultdict(lambda: {'arrival_times': np.ones(max_connections) * np.inf, 'exit_connections': [None] * max_connections})   # trip: [arrival_time_with_1_connection, ..., arrival_time_with_max_connections], incoming_connection
    
    for connection in connections:
        c_dep_stop, c_arr_stop, c_dep_time, c_arr_time, c_trip = connection
        
        # Initialize exit connections of trip with the last temporal connection for this trip: after this trip, it is necessary to exit the line as it does not lead anywhere
        if T[c_trip]['exit_connections'][0] is None:
            T[c_trip]['exit_connections'] = [connection] * max_connections
        
        # PHASE 1: FIND tau_c (arrival times given this connection)
        
        # Arrival times by walk from c_arr_stop 
        # TODO (right now it is just a place-holder)
        tau_1 = np.ones(max_connections) * c_arr_time if c_arr_stop == target_stop else np.inf
        
        # Arrival times by continuing on the same trip
        tau_2 = T[c_trip]['arrival_times']
        
        # Arrival times by moving to another trip
        # TODO add walk time
        tau_3 = search_next_departure(S[c_arr_stop], c_arr_time)[1]['arrival_times']
        tau_3 = shift(tau_3)
        
        # Find minimum per number of changes between tau_1, tau_2 and tau_3
        tau_c = minimize(minimize(tau_1, tau_2), tau_3)

        # PHASE 2: UPDATE S
        
        # arrivals of earliest departure after c_dep_time from stop c_dep_stop
        next_departure_index, next_profile_entry = search_next_departure(S[c_dep_stop], c_dep_time)
        y = next_profile_entry['arrival_times']
        y_departure_connections = next_profile_entry['enter_connections']
        y_exit_connections = next_profile_entry['exit_connections']
        new_min_arrivals, new_enter_connections, new_exit_connections = minimize_with_enter_exit_connections(y, y_departure_connections, tau_c, connection, y_exit_connections, T[c_trip]['exit_connections'])
        if not np.array_equal(y, new_min_arrivals):
            # add another pair before the next one
            S[c_dep_stop].insert(next_departure_index, {'departure_time': c_dep_time, 'arrival_times': new_min_arrivals, 'enter_connections': new_enter_connections, 'exit_connections': new_exit_connections})
        
        # PHASE 3: UPDATE T
        new_min_arrivals, new_exit_connections = minimize_with_stops(T[c_trip]['arrival_times'], T[c_trip]['exit_connections'], tau_c, connection)
        T[c_trip]['arrival_times'] = new_min_arrivals
        T[c_trip]['exit_connections'] = new_exit_connections
        
    return S

In [321]:
def extract_journeys(S, source_stop, source_time, target_stop, changes):
    profiles_source_stop = S[source_stop]
    first_departure_index, _ = search_next_departure(profiles_source_stop, source_time)
    
    paths = set()
    
    for i in range(first_departure_index, len(profiles_source_stop)):
        profile_entry = profiles_source_stop[i]
        enter_connection = profile_entry['enter_connections'][changes]
        exit_connection = profile_entry['exit_connections'][changes]
        current_trip = (enter_connection, exit_connection)
        if enter_connection is None: # If it is not possible to reach target from here in the selected number of changes
            continue
        if changes == 0 and (enter_connection[1] == target_stop or exit_connection[1] == target_stop): # If this connection or this line is directed to the target stop and we do not have more changes to do
            paths.add((enter_connection, exit_connection))
        else: # If we need to exit the line at some point after this connection
            if changes == 0: # If we have run out of the changes budget 
                continue
            next_source_stop = exit_connection[1]
            next_source_time = exit_connection[3]
            paths_from_next = extract_journeys(S, next_source_stop, next_source_time, target_stop, changes - 1)
            paths_from_current = [current_trip + path for path in paths_from_next]
            paths.update(paths_from_current)

    return paths

In [359]:
def describe_journey(path):
    path_len = len(path) // 2 
    for i in range(path_len):
        trip_start = path[i*2]
        trip_end = path[i*2+1]
        print("TRIP {0}. Enter the line at the station {1} at {2} and exit the line at the station {3} at {4}.".format(trip_start[4], trip_start[0], trip_start[2], trip_end[1], trip_end[3]))

In [360]:
def plan_route(source_stop, target_stop, min_departure_time, max_arrival_time, max_connections):
    S = CSA(timetable, target_stop, min_departure_time, max_arrival_time, max_connections + 1)
    all_paths = []
    for changes in range(max_connections + 1):
        paths = extract_journeys(S, source_stop, min_departure_time, target_stop, changes)
        all_paths.extend(paths)
    for path_index, path in enumerate(all_paths):
        print("JORUNEY", path_index)
        describe_journey(path)

In [361]:
paper_conn = [
    ("y", "t", 10, 11, "1"),
    ("z", "t", 9, 12, "2"),
    ("x", "t", 8, 13, "3"),
    ("x", "y", 8, 9, "4"),
    ("s", "z", 7, 8, "5"),
    ("s", "x", 6, 7, "6"),
    ("s", "t", 5, 14, "7")
]
timetable = (None, paper_conn, None, None)

In [362]:
plan_route("s", "t", 4, 15, 3)

JORUNEY 0
TRIP 7. Enter the line at the station s at 5 and exit the line at the station t at 14.
JORUNEY 1
TRIP 5. Enter the line at the station s at 7 and exit the line at the station z at 8.
TRIP 2. Enter the line at the station z at 9 and exit the line at the station t at 12.
JORUNEY 2
TRIP 6. Enter the line at the station s at 6 and exit the line at the station x at 7.
TRIP 4. Enter the line at the station x at 8 and exit the line at the station y at 9.
TRIP 1. Enter the line at the station y at 10 and exit the line at the station t at 11.
